# DB connection 

In [2]:
from pymongo import MongoClient
import matplotlib.pyplot as plt

client = MongoClient() # we run on localhost and default port
db = client['spotify']

# Get Collections

In [3]:
features = db['features']
tracks   = db['tracks']
artists  = db['artists']
albums   = db['albums']


In [4]:
print(features) 
print(tracks  ) 
print(artists ) 
print(albums  ) 

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'spotify'), 'features')
Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'spotify'), 'tracks')
Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'spotify'), 'artists')
Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'spotify'), 'albums')


In [5]:
def display_results(res):
    for entry in res:
        print(entry)

# Query 11
### Average track length per genre

Unfortunately, the "genre" field is related to the artist only, not to the album/track, which would be more accurate.

In [6]:
query11 = [
  {
    '$group': {
      '_id': "$artist_id",
      'averageTrackLengthPerArtist': {'$avg': "$duration_ms"}
    }
  },
  {
    '$lookup': {
      'from': "artists",
      'localField': "_id",
      'foreignField': "id",
      'as': "artistInfo"
    }
  },
  {
    '$unwind': "$artistInfo"
  },
  {
    '$group': {
      '_id': "$artistInfo.genre_0",
      'averageTrackLengthPerGenre': {'$avg': "$averageTrackLengthPerArtist"}
    }
  },
  {
    '$project': {
      '_id': 0,
      'genre': "$_id",
      'averageTrackLengthPerGenre': 1
    }
  },
  {
    '$sort': {
      'averageTrackLengthPerGenre': 1
    }
  }
]

result_query11 = list(albums.aggregate(query11))

Disclaimer: maybe I should not do the first grouping...

In [7]:
result_query11

[{'averageTrackLengthPerGenre': 7882.3, 'genre': 'sound effects'},
 {'averageTrackLengthPerGenre': 29347.57067067067, 'genre': 'ringtone'},
 {'averageTrackLengthPerGenre': 41891.0, 'genre': 'barrelhouse piano'},
 {'averageTrackLengthPerGenre': 59256.0, 'genre': 'cruise'},
 {'averageTrackLengthPerGenre': 60983.1, 'genre': 'birthday'},
 {'averageTrackLengthPerGenre': 66968.38461538461, 'genre': 'hauntology'},
 {'averageTrackLengthPerGenre': 70299.90476190476,
  'genre': 'nordic soundtrack'},
 {'averageTrackLengthPerGenre': 74288.0, 'genre': 'russian viral rap'},
 {'averageTrackLengthPerGenre': 75695.6, 'genre': 'japanoise'},
 {'averageTrackLengthPerGenre': 79003.4, 'genre': 'hypnagogic pop'},
 {'averageTrackLengthPerGenre': 80184.2, 'genre': 'plena uruguaya'},
 {'averageTrackLengthPerGenre': 82725.6, 'genre': 'bachchon ke geet'},
 {'averageTrackLengthPerGenre': 83464.79069767441,
  'genre': 'german romanticism'},
 {'averageTrackLengthPerGenre': 84176.62711864407,
  'genre': 'british elec